In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
import json

## Load Data

In [ ]:
predictions = pd.read_csv('https://fh-public.s3-eu-west-1.amazonaws.com/ml-engineer/input_1.csv')

In [ ]:
predictions.head()

In [ ]:
constraints = requests.get('https://fh-public.s3-eu-west-1.amazonaws.com/ml-engineer/constraints.json').json()
constraints

## Define function to compute 

In [ ]:
from scipy.special import lambertw

def get_accuracy_for(rate, interval):
    """
        Computes the average accuracy for a price given expected 
        rate of change and interval between updates
    """
    return (1 - np.exp(-rate * interval)) / (rate * interval)


def get_ui_for(rate, accuracy):
    """
        function to get update interval given expected rate and accuracy target
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.lambertw.html
    
    """
    W = np.real(lambertw(-np.exp(-1 / accuracy) / accuracy, k=0))
    return ((accuracy * W) + 1) / (accuracy * rate)

## Baseline Solution

In [ ]:
bl = predictions.copy()

In [ ]:
bl['update_interval'] = 60 * get_ui_for(bl.e_change_rate, constraints['target_accuracy'])
bl['rpm'] = 1 / bl.update_interval

bl['e_clicks_per_rpm'] = bl.e_clicks / bl.rpm

bl = bl.sort_values('e_clicks_per_rpm', ascending=False).reset_index(drop=True)

bl['cum_requests'] = bl.rpm.cumsum()

selected = bl[bl.cum_requests < constraints['max_rpm']]

In [ ]:
selected.e_clicks.sum()